# OpenADMET Multi-Task Training with Graph Transformer
This notebook demonstrates how to train a **Graph Transformer** model on the
[OpenADMET](https://openadmet.ghost.io/openadmet-expansionrx-blind-challenge/) multi-task dataset using the `gt-pyg` library.

- Graph Transformer in Pytorch Geometric[gt_pyg](https://github.com/pgniewko/gt-pyg)
- `v1.6.0.0`

> **Note:** This is an exemplary training run for demonstration purposes,
> not a production-grade model.

**Protocol summary:**
- 9 ADMET endpoints trained jointly (sparse labels — not every molecule has every endpoint)
- Single loss term: **MAE** (no weighting, no rescaling)
- 250 epochs, **cosine annealing** LR schedule (no warmup)
- Track best macro-averaged MAE on the validation set
- Evaluate on the held-out test set (public leaderboard + private split)

> **Loss caveat:** The plain MAE loss treats all endpoints and all datapoints
> uniformly. Endpoints with more observed labels or larger variance therefore
> have disproportionate influence on the gradient. A production model would
> use per-task normalisation (e.g. RAE) or a multi-objective loss.

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch_geometric.loader import DataLoader
from scipy.stats import spearmanr, kendalltau
from sklearn.metrics import r2_score
from copy import deepcopy

from gt_pyg.data import get_tensor_data, get_atom_feature_dim, get_bond_feature_dim
from gt_pyg.nn import GraphTransformerNet

## 2. Configuration

In [ ]:
# Paths
TRAIN_CSV = "data/train-set/expansion_log_data_train.csv"
TEST_CSV = "data/test-set/expansion_data_test_full_lb_flag.csv"

# Model
HIDDEN_DIM = 128
NUM_GT_LAYERS = 4
NUM_HEADS = 8
DROPOUT = 0.1

# Training
EPOCHS = 250
BATCH_SIZE = 256
LR = 1e-3
WEIGHT_DECAY = 1e-5

SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Device: {DEVICE}")

## 3. Load and Split Data

The training set contains log-transformed ADMET endpoint values. Many values are
missing — this is a **sparse multi-task** problem. We perform a random 80/20
train/validation split.

In [ ]:
log_train_df = pd.read_csv(TRAIN_CSV)

ID_COLS = {"SMILES", "Molecule Name"}
ENDPOINTS = [c for c in log_train_df.columns if c not in ID_COLS]
NUM_TASKS = len(ENDPOINTS)

print(f"Total molecules: {len(log_train_df)}")
print(f"Endpoints ({NUM_TASKS}): {ENDPOINTS}")
print(f"\nLabel coverage per endpoint:")
for ep in ENDPOINTS:
    n = log_train_df[ep].notna().sum()
    print(f"  {ep}: {n} / {len(log_train_df)} ({100*n/len(log_train_df):.1f}%)")

In [ ]:
# Shuffle and split 80/20
df = log_train_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
n_train = int(0.8 * len(df))
tr_df = df.iloc[:n_train].copy()
va_df = df.iloc[n_train:].copy()

print(f"Train: {len(tr_df)}, Validation: {len(va_df)}")

## 4. Build PyG Datasets and DataLoaders

`get_tensor_data` converts SMILES + labels into PyG `Data` objects with
atom/bond features, GNM positional encodings, and multi-task label tensors.
Missing labels are encoded as `NaN` with an accompanying binary mask (`y_mask`).

In [ ]:
def build_dataset(df, endpoints):
    """Convert a DataFrame into a list of PyG Data objects."""
    smiles = df["SMILES"].tolist()
    labels = df[endpoints].values.tolist()  # list of lists, NaN for missing
    return get_tensor_data(smiles, labels)

print("Building training set...")
tr_dataset = build_dataset(tr_df, ENDPOINTS)
print("Building validation set...")
va_dataset = build_dataset(va_df, ENDPOINTS)

tr_loader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True)
va_loader = DataLoader(va_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nTrain batches: {len(tr_loader)}, Val batches: {len(va_loader)}")
print(f"Node features: {tr_dataset[0].x.shape[1]}, Edge features: {tr_dataset[0].edge_attr.shape[1]}")

## 5. Create Model, Optimizer, and Scheduler

We use `GraphTransformerNet` with a variational readout head. During evaluation
the model returns the deterministic mean (`mu`). The LR follows a **cosine
annealing** schedule from `LR` down to 0 over 250 epochs.

In [ ]:
node_dim = get_atom_feature_dim()
edge_dim = get_bond_feature_dim()

model = GraphTransformerNet(
    node_dim_in=node_dim,
    edge_dim_in=edge_dim,
    hidden_dim=HIDDEN_DIM,
    num_gt_layers=NUM_GT_LAYERS,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    num_tasks=NUM_TASKS,
).to(DEVICE)

print(f"Parameters: {model.num_parameters():,}")

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=0)

## 6. Training and Evaluation Functions

The loss is **masked MAE**: for each sample, only endpoints with observed labels
contribute to the loss. This handles the sparse multi-task structure naturally.

Evaluation computes the **5 official challenge metrics** per endpoint:
MAE, RAE, R², Spearman $\rho$, Kendall $\tau$.

In [ ]:
def masked_mae_loss(pred, target, mask):
    """MAE loss computed only over observed (non-NaN) labels."""
    diff = torch.abs(pred - target) * mask
    count = mask.sum()
    if count == 0:
        return torch.tensor(0.0, device=pred.device)
    return diff.sum() / count


def train_epoch(model, loader, optimizer, device):
    """Run one training epoch. Returns average loss."""
    model.train()
    total_loss = 0.0
    n_batches = 0

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )

        target = batch.y
        mask = batch.y_mask
        # Replace NaN in target with 0 to avoid NaN in loss
        target = torch.nan_to_num(target, nan=0.0)

        loss = masked_mae_loss(pred, target, mask)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        n_batches += 1

    return total_loss / max(n_batches, 1)


def _official_metrics(y_true_1d, y_pred_1d):
    """Compute the 5 official challenge metrics for a single endpoint."""
    y = np.asarray(y_true_1d).ravel()
    p = np.asarray(y_pred_1d).ravel()
    m = np.isfinite(y) & np.isfinite(p)
    y = y[m]
    p = p[m]
    if y.size == 0:
        return {
            "MAE": np.nan,
            "RAE": np.nan,
            "R2": np.nan,
            "Spearman R": np.nan,
            "Kendall's Tau": np.nan,
        }
    mae = float(np.mean(np.abs(y - p)))
    denom = np.mean(np.abs(y - np.mean(y))) if y.size > 0 else np.nan
    rae = float(mae / denom) if denom and np.isfinite(denom) and denom > 0 else np.nan
    r2 = float(r2_score(y, p)) if (np.nanstd(y) > 0) else np.nan
    if np.nanstd(p) < 1e-4:
        spr = np.nan
        ktau = np.nan
    else:
        spr = float(spearmanr(y, p)[0])
        ktau = float(kendalltau(y, p)[0])
    return {
        "MAE": mae,
        "RAE": rae,
        "R2": r2,
        "Spearman R": spr,
        "Kendall's Tau": ktau,
    }


@torch.no_grad()
def evaluate(model, loader, device, endpoints):
    """Evaluate model. Returns (metrics_dict, preds, targets, masks).

    metrics_dict[endpoint] has 5 official metrics + N.
    metrics_dict["Average"] has macro-averaged values (nanmean).
    """
    model.eval()
    all_preds = []
    all_targets = []
    all_masks = []

    for batch in loader:
        batch = batch.to(device)
        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )
        all_preds.append(pred.cpu())
        all_targets.append(batch.y.cpu())
        all_masks.append(batch.y_mask.cpu())

    preds = torch.cat(all_preds, dim=0).numpy()
    targets = torch.cat(all_targets, dim=0).numpy()
    masks = torch.cat(all_masks, dim=0).numpy()

    metrics = {}
    for i, ep in enumerate(endpoints):
        valid = masks[:, i] > 0.5
        if valid.sum() < 2:
            continue
        y = targets[valid, i]
        p = preds[valid, i]
        official = _official_metrics(y, p)
        official["N"] = int(valid.sum())
        metrics[ep] = official

    # Macro averages (nanmean over endpoints)
    keys = ["MAE", "RAE", "R2", "Spearman R", "Kendall's Tau"]
    avg = {}
    for k in keys:
        vals = [metrics[ep][k] for ep in metrics if ep != "Average"]
        avg[k] = float(np.nanmean(vals)) if vals else np.nan
    metrics["Average"] = avg

    return metrics, preds, targets, masks

## 7. Training Loop

We train for 250 epochs, tracking the **best macro-averaged MAE** on the
validation set. The best model weights are stored in memory.

In [ ]:
best_val_mae = float("inf")
best_model_state = None
best_epoch = -1

history = {
    "epoch": [], "train_loss": [], "lr": [],
    "val_mae": [], "val_rae": [], "val_r2": [],
    "val_spearman": [], "val_kendall": [],
    # Train metrics recorded at logged epochs only
    "train_mae": [], "train_rae": [], "train_r2": [],
    "train_spearman": [], "train_kendall": [],
    "logged_epoch": [],
}

KT_KEY = "Kendall's Tau"

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(model, tr_loader, optimizer, DEVICE)
    scheduler.step()

    val_metrics_ep, _, _, _ = evaluate(model, va_loader, DEVICE, ENDPOINTS)

    val_avg = val_metrics_ep["Average"]
    val_mae = val_avg["MAE"]
    lr_now = optimizer.param_groups[0]["lr"]

    history["epoch"].append(epoch)
    history["train_loss"].append(train_loss)
    history["lr"].append(lr_now)
    history["val_mae"].append(val_mae)
    history["val_rae"].append(val_avg["RAE"])
    history["val_r2"].append(val_avg["R2"])
    history["val_spearman"].append(val_avg["Spearman R"])
    history["val_kendall"].append(val_avg[KT_KEY])

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_epoch = epoch
        best_model_state = deepcopy(model.state_dict())

    if epoch % 10 == 0 or epoch == 1:
        tr_metrics_ep, _, _, _ = evaluate(model, tr_loader, DEVICE, ENDPOINTS)
        tr_avg = tr_metrics_ep["Average"]

        history["logged_epoch"].append(epoch)
        history["train_mae"].append(tr_avg["MAE"])
        history["train_rae"].append(tr_avg["RAE"])
        history["train_r2"].append(tr_avg["R2"])
        history["train_spearman"].append(tr_avg["Spearman R"])
        history["train_kendall"].append(tr_avg[KT_KEY])

        print(
            f"Epoch {epoch:3d}/{EPOCHS} | "
            f"Loss: {train_loss:.4f} | "
            f"Train MAE={tr_avg['MAE']:.3f} RAE={tr_avg['RAE']:.3f} "
            f"R2={tr_avg['R2']:.3f} rho={tr_avg['Spearman R']:.3f} "
            f"tau={tr_avg[KT_KEY]:.3f} | "
            f"Val MAE={val_avg['MAE']:.3f} RAE={val_avg['RAE']:.3f} "
            f"R2={val_avg['R2']:.3f} rho={val_avg['Spearman R']:.3f} "
            f"tau={val_avg[KT_KEY]:.3f} | "
            f"LR: {lr_now:.2e}"
        )

print(f"\nBest val MAE: {best_val_mae:.4f} at epoch {best_epoch}")

## 8. Restore Best Model and Print Final Stats

Load the best checkpoint (by validation MAE) and report per-endpoint metrics
on both train and validation sets.

In [ ]:
model.load_state_dict(best_model_state)

tr_metrics, _, _, _ = evaluate(model, tr_loader, DEVICE, ENDPOINTS)
va_metrics, _, _, _ = evaluate(model, va_loader, DEVICE, ENDPOINTS)

header = (
    f"{'Endpoint':<22} "
    f"{'Train MAE':>10} {'Val MAE':>10} "
    f"{'Train RAE':>10} {'Val RAE':>10} "
    f"{'Train R2':>10} {'Val R2':>10} "
    f"{'Train rho':>10} {'Val rho':>10} "
    f"{'Train tau':>10} {'Val tau':>10} "
    f"{'N_val':>7}"
)
print(header)
print("-" * len(header))

KT_KEY = "Kendall's Tau"

for ep in ENDPOINTS:
    if ep in tr_metrics and ep in va_metrics:
        tm = tr_metrics[ep]
        vm = va_metrics[ep]
        print(
            f"{ep:<22} "
            f"{tm['MAE']:>10.4f} {vm['MAE']:>10.4f} "
            f"{tm['RAE']:>10.4f} {vm['RAE']:>10.4f} "
            f"{tm['R2']:>10.4f} {vm['R2']:>10.4f} "
            f"{tm['Spearman R']:>10.4f} {vm['Spearman R']:>10.4f} "
            f"{tm[KT_KEY]:>10.4f} {vm[KT_KEY]:>10.4f} "
            f"{vm['N']:>7d}"
        )

print("-" * len(header))
ta = tr_metrics["Average"]
va = va_metrics["Average"]
print(
    f"{'Average':<22} "
    f"{ta['MAE']:>10.4f} {va['MAE']:>10.4f} "
    f"{ta['RAE']:>10.4f} {va['RAE']:>10.4f} "
    f"{ta['R2']:>10.4f} {va['R2']:>10.4f} "
    f"{ta['Spearman R']:>10.4f} {va['Spearman R']:>10.4f} "
    f"{ta[KT_KEY]:>10.4f} {va[KT_KEY]:>10.4f}"
)

## 9. Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ---- (0,0) Loss curves (LOG SCALE) ----
ax = axes[0, 0]
ax.semilogy(history["epoch"], history["train_loss"], label="Train Loss", alpha=0.8)
# Compute val loss from val_mae as proxy (we don't track val loss separately)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss (log scale)")
ax.set_title("Training Loss")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (0,1) RAE (capped at 1.0) ----
ax = axes[0, 1]
ax.plot(history["epoch"], history["val_rae"], "b-", alpha=0.8, label="Val RAE")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_rae"], "b--", alpha=0.5, label="Train RAE")
best_rae = min(history["val_rae"])
ax.axhline(best_rae, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_rae:.3f}")
ax.set_ylim(0, 1.0)
ax.set_xlabel("Epoch")
ax.set_ylabel("RAE")
ax.set_title("RAE (capped at 1.0)")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (1,0) R² (capped at 1.0) ----
ax = axes[1, 0]
ax.plot(history["epoch"], history["val_r2"], "g-", alpha=0.8, label="Val R²")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_r2"], "g--", alpha=0.5, label="Train R²")
best_r2 = max(history["val_r2"])
ax.axhline(best_r2, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_r2:.3f}")
ax.set_ylim(-0.1, 1.0)
ax.set_xlabel("Epoch")
ax.set_ylabel("R²")
ax.set_title("R² (capped at 1.0)")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (1,1) Kendall τ ----
ax = axes[1, 1]
ax.plot(history["epoch"], history["val_kendall"], "m-", alpha=0.8, label="Val τ")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_kendall"], "m--", alpha=0.5, label="Train τ")
best_tau = max(history["val_kendall"])
ax.axhline(best_tau, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_tau:.3f}")
ax.set_xlabel("Epoch")
ax.set_ylabel("Kendall τ")
ax.set_title("Kendall τ")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Bar Plots — Train vs Validation MAE and R²

In [ ]:
# Build metrics dataframe for bar plots
metric_records = []
for ep in ENDPOINTS:
    tm = tr_metrics.get(ep, {})
    vm = va_metrics.get(ep, {})
    metric_records.append({"Assay": ep, "Split": "Train", "R2": tm.get("R2", np.nan), "MAE": tm.get("MAE", np.nan)})
    metric_records.append({"Assay": ep, "Split": "Val", "R2": vm.get("R2", np.nan), "MAE": vm.get("MAE", np.nan)})

metrics_df = pd.DataFrame(metric_records)

# Order assays by validation MAE ascending
order_by_val = (
    metrics_df[metrics_df["Split"] == "Val"]
    .sort_values("MAE")["Assay"]
    .tolist()
)

sns.set_style("whitegrid")

# ---- MAE bar plot ----
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(x="Assay", y="MAE", hue="Split", data=metrics_df, ax=ax, order=order_by_val)
ax.set_ylabel("MAE (log-space)")
ax.set_title("Training vs Validation MAE per Endpoint")
ax.set_xticklabels([t.get_text().replace("_", "\n") for t in ax.get_xticklabels()])
plt.tight_layout()
plt.show()

# ---- R² bar plot ----
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(x="Assay", y="R2", hue="Split", data=metrics_df, ax=ax, order=order_by_val)
ax.set_ylim(-1.1, 1.0)
ax.set_ylabel("$R^2$")
ax.set_title("Training vs Validation R² per Endpoint")
ax.set_xticklabels([t.get_text().replace("_", "\n") for t in ax.get_xticklabels()])
plt.tight_layout()
plt.show()

## 11. Test Set Prediction & Submission

Generate predictions on the held-out test set and save as `submission.csv`.
The model predicts in **log-space** (matching the training labels), and we
inverse-transform to **original assay units** for the submission file.

In [ ]:
# --- Load test set ---
test_df = pd.read_csv(TEST_CSV)
print(f"Test samples: {len(test_df)}")
print(f"Columns: {test_df.columns.tolist()}")

smiles_test = test_df["SMILES"].tolist()

# --- Build PyG dataset (inference — no labels) ---
print("\nFeaturising test SMILES...")
test_dataset = get_tensor_data(smiles_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Run inference ---
model.eval()
test_preds_list = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(DEVICE)
        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )
        test_preds_list.append(pred.cpu().numpy())

test_preds_log = np.concatenate(test_preds_list, axis=0)  # [N, 9] in log-space
print(f"Predictions shape: {test_preds_log.shape}")

# --- Inverse-transform from log-space to original assay units ---
# Mapping: log-space endpoint name -> (submission column name, transform)
INVERSE_MAP = {
    "LogD":             ("LogD",                           lambda x: x),         # already log-scale
    "LogS":             ("KSOL",                           lambda x: 10**x),
    "Log_HLM_CLint":    ("HLM CLint",                     lambda x: 10**x),
    "Log_MLM_CLint":    ("MLM CLint",                     lambda x: 10**x),
    "Log_Caco_Papp_AB": ("Caco-2 Permeability Papp A>B",  lambda x: 10**x),
    "Log_Caco_ER":      ("Caco-2 Permeability Efflux",    lambda x: 10**x),
    "Log_Mouse_PPB":    ("MPPB",                           lambda x: 10**x),
    "Log_Mouse_BPB":    ("MBPB",                           lambda x: 10**x),
    "Log_Mouse_MPB":    ("MGMB",                           lambda x: 10**x),
}

# Build submission DataFrame
sub_df = test_df[["Molecule Name", "SMILES"]].copy()

for i, ep_log in enumerate(ENDPOINTS):
    col_name, transform = INVERSE_MAP[ep_log]
    sub_df[col_name] = transform(test_preds_log[:, i])

sub_df.to_csv("submission.csv", index=False)
print(f"\nSaved submission.csv with {len(sub_df)} rows")
print(f"Columns: {sub_df.columns.tolist()}\n")

# --- Show head and summary ---
print("Head:")
print(sub_df.head().to_string())

print("\nPrediction range summary (original scale):")
for col in sub_df.columns:
    if col not in ("Molecule Name", "SMILES"):
        vals = sub_df[col]
        print(f"  {col:<35s}  min={vals.min():.4f}  max={vals.max():.4f}  mean={vals.mean():.4f}")